In [31]:
import os
import getpass
import random
from openai import OpenAI

In [2]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

Enter your OpenAI API key: ··········


In [32]:
def score_text():
    s = round(random.uniform(0.40, 0.95), 2)
    label = "go" if s > 0.80 else "no-go"
    return s, label

In [33]:
def generate_project_summary(details):
    prompt = f"""
Create a clean multi-line project summary based on the user's details.

Important formatting instructions:
- Do NOT use bold text, markdown symbols, or asterisks.
- Provide the output in plain text only.
- Format exactly like this:

Title: <short clear title>

Description:
<3 to 5 separate lines explaining the project>

User details:
{details}
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
    )

    return response.choices[0].message.content.strip()

In [34]:
def improve_summary(prev_summary, new_details):
    prompt = f"""
Improve the previous project summary using the additional details.

Rules:
- Do NOT use markdown or bold or asterisks.
- Keep Title: on top.
- Keep Description: below it on separate lines.
- Break the description into 3–6 readable lines.
- Maintain clean, academic, and technical language.

Previous summary:
{prev_summary}

New user details to include:
{new_details}
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.8,
    )

    return response.choices[0].message.content.strip()


In [38]:
while True:

    print("\nEnter project details:\n")
    project_details = input()

    # FIRST 4 SCORES
    for i in range(1, 4):
        s, label = score_text()
        print(f"\n{i} {label} {s}")

    print(
        "\nThe framework is currently generating very generic response. "
        "Enter more specific details about your work, experience, projects, or yourself\n"
    )

    # FIRST SUMMARY FROM LLM
    best_response = generate_project_summary(project_details)
    best_score = 0.55

    print("This is the best response so far:\n")
    print(best_response)

    # IMPROVEMENT LOOP
    while True:
        print("\nType 'keep' to keep the best response and terminate the loop")
        user_more = input("\nEnter further details here:\n")

        if user_more.lower() == "keep":
            break

        new_score, new_label = score_text()
        print(f"\n5 {new_label} {new_score}")

        new_response = improve_summary(best_response, user_more)

        if new_score > best_score:
            best_score = new_score
            best_response = new_response

        print("\n" + best_response)

    nxt = input("\nDo you want to add another project? (y/n): ").lower()
    if nxt != "y":
        break

print("\nAll projects recorded successfully.\n")


Enter project details:

did a regression project where I used multiple linear regression and random forest regressor to predict house prices using features like area, location and number of rooms.

1 no-go 0.66

2 no-go 0.58

3 no-go 0.43

The framework is currently generating very generic response. Enter more specific details about your work, experience, projects, or yourself

This is the best response so far:

Title: House Price Prediction Project

Description:  
This project involved predicting house prices using advanced regression techniques.  
Multiple linear regression and random forest regressor models were utilized for accurate predictions.  
Key features included area, location, and number of rooms, which significantly influenced pricing.  
The analysis aimed to provide insights into the real estate market dynamics.

Type 'keep' to keep the best response and terminate the loop

Enter further details here:
performed correlation analysis to identify key factors affecting price